In [ ]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np

df=pd.read_csv("/content/train.csv")

df.head()

df.isnull().sum()

df.duplicated().sum()

from tensorflow.keras.layers import TextVectorization

x=df['comment_text']
y=df[df.columns[2:]].values

MAX_FEATURES=200000

vectorizer=TextVectorization(max_tokens=MAX_FEATURES,output_sequence_length=1800,output_mode='int')

vectorizer.adapt(x.values)

vectorized_text=vectorizer(x.values)

dataset=tf.data.Dataset.from_tensor_slices((vectorized_text,y))
dataset=dataset.cache()
dataset=dataset.shuffle(160000)
dataset=dataset.batch(16)
dataset=dataset.prefetch(8)

train=dataset.take(int(len(dataset)*.7))
val=dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2))
test=dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))

from tensorflow.keras.layers import LSTM,Dropout,Bidirectional,Dense,Embedding
from tensorflow.keras.models import Sequential

model=Sequential()

model.add(Embedding(MAX_FEATURES+1,32))
model.add(Bidirectional(LSTM(32,activation='tanh')))
model.add(Dense(128,activation='relu'))
model.add(Dense(256,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(6,activation='sigmoid'))

model.compile(loss='BinaryCrossentropy',optimizer='Adam')

model.summary()

history=model.fit(train,epochs=1,validation_data=val)

from matplotlib import pyplot as plt

plt.figure(figsize=(8,5))
pd.DataFrame(history.history).plot()
plt.show()

input_text=vectorizer("You freaking suck!")

res=model.predict(np.expand_dims(input_text,0))

print(res)

df.columns[2:]
